In [10]:
########################################################################################################################
# This script runs RiskPath Transformer modeling.
########################################################################################################################

In [5]:
########################################################################################################################
# Import packages
########################################################################################################################
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import warnings
from captum.attr import GradientShap
from _Helper_Scripts.ablation import ablate
from _Helper_Scripts.result_organizer import optimize_width
from _Helper_Scripts.transformer_dense import EarlyStopping, Transformer_Classifier, train_tf, eval_tf
from itertools import product
from sklearn.model_selection import train_test_split, StratifiedKFold
from time import time
from typing import Optional
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
np.random.seed(42)
torch.manual_seed(42)
print('Packages loaded')

Packages loaded


In [6]:
########################################################################################################################
# USER_SPECIFIC SETTING
# C_LIST: A list of different numbers of feature encounteres to be included
# D_LIST: A list of different maximum widths of the look-back window in days
# CSL_LIST: A list of boolean indicating whether the perform cost-sensitive learning (CSL)
# IMPUTE_LIST: A list of strings representing the imputation methods adopted
# ABLATION: Boolean. False for pre-ablation modeling and True for post-ablation modeling
# CS_GRID: A list of cost multipliers to penalize false positives
# RP_GRID: A list of integers representing the number of hidden units (model width) for model's structural optimization
# N_FOLDS: An integer representing the number of folds for cross-validation
# N_LAYERS: An integer representing the number of hidden layers in the model
# N_HEADS: An integer representing the number of attention heads in the transformer model
# N_UNITS: An integer representing the number of hidden units in each encoder layer
########################################################################################################################
C_LIST: list[int] = [2, 3, 4]                   # Number of encounters (Default: [2, 3, 4])
D_LIST: list[int] = [60, 120, 180]              # Width of look-back window (Default: [60, 120, 180])
CSL_LIST: list[bool] = [False, True]            # Whether to perfrom cost-sensitive learning (Default: [False, True])
IMPUTE_LIST: list[str] = ['Zero', 'Mean', 'Median'] 
                                                # Method of imputation (Default: ['Zero', 'Mean', 'Median']
ABLATION: bool = False                          # Whether to perform model ablation (Default: False. Use True only after False)
CS_GRID: list[int] = [2, 3, 4]                  # Grid for cost multipliers (Default: [2, 3, 4])
RP_GRID: list[int] = list(range(128, 1025, 128))
                                                # Grid for model width for model's structural optimization 
                                                # (Default: list(range(128, 1025, 128)))
N_FOLDS: int = 5                                # Number of folds for cross-validation (Default: 5)
N_LAYERS: int = 2                               # Numbre of hidden layers (Default: 3)
N_HEADS: int = 4                                # Number of attention heads (Default: 4)
N_UNITS: int = 128                              # Number of hidden units in each encoder layer (Default: 128)

In [7]:
########################################################################################################################
# USER_SPECIFIC SETTING
# IN_DIR_PATH: Path of the input directory storing the organized datasets for modeling
# (created in P06_Point_Data_Preparation.ipynb)
########################################################################################################################
IN_DIR_PATH: str = '../00_Data/02_Processed_Data/Longitudinal_Model_Data/'

In [8]:
########################################################################################################################
# 1. Define a function to extract the SHAP feature importance
########################################################################################################################
def get_shap(model, X_train, y_train, X_test, feature_names):

    # Define the type of the input datasets
    X_train = np.asarray(X_train).astype(np.float32, copy=False)
    X_test = np.asarray(X_test).astype(np.float32, copy=False)
    y_train = np.asarray(y_train).ravel().astype(np.int64, copy=False)

    # Define the baseline and median feature values for positive and negative cases separately
    m0 = np.nanmedian(X_train[y_train == 0], axis=0)
    m1 = np.nanmedian(X_train[y_train == 1], axis=0)
    L, F = X_test.shape[1], X_test.shape[2]
    X_baseline = torch.tensor(np.stack([m0, m1]).astype(np.float32))
    assert X_baseline.shape == (2, L, F)

    # Prepare the explanation model
    model.eval()
    M = GradientShap(forward_func=model)

    # Fit the explanation model (with batching)
    attrs = []
    bs = 512
    for i in range(0, len(X_test), bs):
        x = torch.tensor(X_test[i:i+bs], dtype=torch.float32)
        a = M.attribute(inputs=x, baselines=X_baseline, n_samples=16)
        attrs.append(a.detach())

    # Obtain the shap values and its mean-absolute values (across patients and timestamps)
    shap = torch.cat(attrs, dim=0).numpy() 
    mean_abs_shap = np.abs(shap).mean(axis=(0, 1))

    # Format the mean-absolute shap values as a pandas.DataFrame
    df_shap = pd.DataFrame({'Feature': feature_names,
                            'MeanAbsSHAP': mean_abs_shap}).sort_values(by='MeanAbsSHAP', ascending=False)

    # Return shap (raw) and df_shap (summarized)
    return df_shap, shap

In [9]:
########################################################################################################################
# 2. Define a function to load the data
########################################################################################################################
def data_load(C: int,
              D: int,
              impute: str,
              feats: Optional[list[str]] = None):

    # Specify the paths of the datasets
    dir_path: str = os.path.join(IN_DIR_PATH, f'{C}_encounters_{D}_days/', f'{impute}/')
    X_train_path: str = f'{dir_path}X_train.npy'
    X_test_path: str = X_train_path.replace('train', 'test')
    y_train_path: str = f'{dir_path}y_train.npy'
    y_test_path: str = y_train_path.replace('train', 'test')
    feat_name_path: str = f'{dir_path}Feature_Names.csv'

    # Load the datasets
    X_train: np.ndarray = np.load(X_train_path, allow_pickle=True)
    X_test: np.ndarray = np.load(X_test_path, allow_pickle=True)
    y_train: np.ndarray = np.load(y_train_path, allow_pickle=True)
    y_test: np.ndarray = np.load(y_test_path, allow_pickle=True)
    feat_names: list[str] = pd.read_csv(feat_name_path)['Features'].to_list()

    # Specify the name of the dataset
    data_str: str = f'{C}_encounters_{D}_days_{impute}'

    # Truncate the feature datasets if needed (for ablation purposes)
    if feats is not None:
        assert set(feats).issubset(feat_names)
        idxs: list[int] = [feat_names.index(f) for f in feats]
        X_train = X_train[:, :, idxs]
        X_test = X_test[:, :, idxs]
        data_str += '_Ablated'

    # Specify the features being used in the dataset
    feats_out = feat_names if feats is None else feats

    # Return the datasets, features, and the name of the dataset    
    return X_train, X_test, y_train, y_test, feats_out, data_str

In [16]:
########################################################################################################################
# 3. Define an overall function to run transformer with 5-fold cross-validation
########################################################################################################################
def run_tf_riskpath(C: int,
                    D: int,
                    impute: str,
                    feats: Optional[list[str]] = None,
                    csl: bool = True):                  # csl: cost-sensitive learning
    
    # Load the dataset
    X_train, X_test, y_train, y_test, feat_names, data_str = data_load(C=C, D=D, impute=impute, feats=feats)
    prevalence: float = np.mean(y_train)

    # Logging
    log_head: str = f'[C={C}; D={D}; impute={impute}; CSL={csl}] '    
    if csl:
        data_str += '_CSL'

    # Define the starting time of the RiskPath modeling
    t0_out: float = time()

    # Prepare a dictionary to store the performance statistics (evaluated at the cross-validation sets)
    if csl:
        result_dict: dict[tuple[int, int], list] = {(rp_width, cost_mul): [] 
                                                    for rp_width, cost_mul in product(RP_GRID, CS_GRID)}
    else:
        result_dict: dict[int, list] = {rp_width: [] for rp_width in RP_GRID}

    # Loop over the full grid
    full_grid = product(RP_GRID, CS_GRID) if csl else RP_GRID
    for grid_element in full_grid:
        if csl:
            width, cost_mul = grid_element
        else:
            width, cost_mul = grid_element, None 

        # Define the folds for cross-validation
        skf = StratifiedKFold(n_splits=N_FOLDS, random_state=42, shuffle=True)

        # Loop over the k-fold
        for k_idx, (train, val) in enumerate(skf.split(X_train, y_train), 1):
            X_train_cur, X_val_cur = np.take(X_train, train, axis=0), np.take(X_train, val, axis=0)
            y_train_cur, y_val_cur = np.take(y_train, train), np.take(y_train, val)

            # Logging
            log_head_sub = f'{log_head}<Width={width}>; CS_Ratio={cost_mul}; Fold={k_idx}/{N_FOLDS} '

            # Create the transformer model
            M = Transformer_Classifier(n_feat=X_train_cur.shape[2], n_timestamps=C,     
                                       d_model=width,
                                       n_layers=N_LAYERS, n_units=N_UNITS, n_heads=N_HEADS)
            M.init_Xavier_weights()

            # In-fold fitting
            t0: float = time()
            M = train_tf(M, X_train_cur, y_train_cur, X_val_cur, y_val_cur, cost_mul=cost_mul)
            train_elapsed: float = round(time() - t0, 3)
            print(f'{log_head_sub}Took {train_elapsed} seconds.')

            # In-fold evaluation
            val_result: dict[str, float] = eval_tf(M, X_val_cur, y_val_cur, prefix='Val_')[0]
            val_result = {'Width': width, 'Cost_multiplier': cost_mul} | {k: v for k, v in val_result.items() if not 'LIST' in k}
            result_dict[grid_element].append(val_result)
            print('.'*120)

        # Store the averaged cross-validation performance statistics
        result_dict[grid_element] = pd.DataFrame(pd.DataFrame.from_records(result_dict[grid_element]).mean()).T

    # Logging and return result_dict    
    print(f'{log_head}RiskPath optimization completed in {round(time() - t0_out, 3)} seconds.')
    return result_dict

In [17]:
########################################################################################################################
# 4. Define an overall function to generate the modeling result
########################################################################################################################
def run_rp_pipeline(C: int,
                    D: int,
                    impute: str,
                    feats: Optional[list[str]] = None,
                    csl: bool = True):

    # Logging
    log_head: str = f'[C={C}; D={D}; impute={impute}, CSL={csl}] '    

    # Run RiskPath width optimization
    rp_results: dict = run_tf_riskpath(C=C, D=D, impute=impute, feats=feats, csl=csl)

    # Identify the optimized width which has the highest averaged validation metric
    metric: str = 'Val_Recall@99Spec' if csl else 'Val_AUPRC'
    df_rp: pd.DataFrame = pd.concat(rp_results.values()).reset_index(drop=True)
    opt_width: int = int(df_rp.loc[df_rp[metric].idxmax(), 'Width'])
    print(f'{log_head}Optimized model width={opt_width}')
    if csl:
        opt_cost_mul: int = int(df_rp.loc[df_rp[metric].idxmax(), 'Cost_multiplier'])
        print(f'{log_head}Optimized cost multiplier={opt_cost_mul}')
    else:
        opt_cost_mul = None

    # Re-load the dataset and form 8:2 stratified partition for training and validation
    X_train, X_test, y_train, y_test, feat_names, data_str = data_load(C=C, D=D, impute=impute, feats=feats)
    X_train_cur, X_val_cur, y_train_cur, y_val_cur = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)
    if csl:
        data_str += '_CSL'

    # Create and refit the model with the optimized width (and optimal cost ratio if CSL)
    M = Transformer_Classifier(n_feat=X_train_cur.shape[2], n_timestamps=C,     
                               d_model=opt_width,
                               n_layers=N_LAYERS, n_units=N_UNITS, n_heads=N_HEADS)
    M.init_Xavier_weights()
    t0: float = time()
    M = train_tf(M, X_train_cur, y_train_cur, X_val_cur, y_val_cur, cost_mul=opt_cost_mul)
    train_elapsed: float = round(time() - t0, 3)
    print(f'{log_head}Training took {train_elapsed} seconds.')

    # Held-out evaluation
    train_result: dict[str, float] = eval_tf(M, X_train, y_train, prefix='Train_')[0]
    test_result, test_elapsed = eval_tf(M, X_test, y_test, prefix='Test_')
    print(f'{log_head}Basic evaluation completed.')

    # Organize the results
    final_result: dict[str, float] = {'Algorithm': 'RiskPath_Transformer',
                                      'Model_Width': opt_width,
                                      '#Encounters': C,
                                      'LookBackDays': D,
                                      'Impute': impute,
                                      'Experiment_Name': data_str,
                                      'Features': 'All' if feats is None else 'Ablated',
                                      'Cost_Ratio': opt_cost_mul if csl else 'NONE',
                                      'Train_Sample_Size': X_train.shape[0],
                                      'Test_Sample_Size': X_test.shape[0],
                                      'Feature_Size': X_train.shape[2],
                                      'Prevalence': np.round(np.mean(y_train), 3),
                                      'Training_Time_Seconds': train_elapsed,
                                      'Test_Time_Seconds': test_elapsed}
    final_result |= test_result | train_result    
    t0_shap: float = time()
    df_shap, np_shap = get_shap(M, X_train, y_train, X_test, feat_names)  
    t1_shap: float = time()
    print(f'{log_head}SHAP computation completed in {round(t1_shap - t0_shap)} seconds.')
    y_prob: np.ndarray = torch.sigmoid(M(torch.tensor(np.array(X_test, dtype=np.float32))).view(-1)).detach().cpu().numpy()
    df_y: pd.DataFrame = pd.DataFrame({'y_test': y_test, 'y_prob': y_prob})    
    return final_result, df_shap, np_shap, df_rp, df_y

In [ ]:
########################################################################################################################
# 5. Run all the experiments (and store results in the current working directory)
########################################################################################################################
all_results: list[dict] = []
out_dir_path: str = 'RiskPath_Results/' if not ABLATION else 'RiskPath_Results_Ablated/'
os.makedirs(out_dir_path, exist_ok=True)

for exp_idx, (C, D, csl, impute) in enumerate(product(C_LIST, D_LIST, CSL_LIST, IMPUTE_LIST), 1):

    # Logging 
    log_head: str = f'[Exp {exp_idx}. C={C}; D={D}; impute={impute}; CSL={csl}; Ablation={ABLATION}] '    
    print(f'{log_head}Starting experiment...')

    # Perform feature ablation if needed
    if ABLATION:
        print(f'{log_head}Performing ablation...')
        shap_filename: str = f'{C}_encounters_{D}_days_{impute}{"_CSL" if csl else ""}.csv'
        df_shap: pd.DataFrame = pd.read_csv(f'RiskPath_Results/SHAP/{shap_filename}')
        elbow_dir_path: str = f'{out_dir_path}/SHAP/Elbow_Images/'
        os.makedirs(elbow_dir_path, exist_ok=True)
        elbow_filename: str = elbow_dir_path + shap_filename.replace('.csv', '_elbow.png')
        feats: list[str] = ablate(df_shap, 'MeanAbsSHAP', elbow_filename)
    else:
        feats = None

    # Run RiskPath transformer modeling
    cur_result, df_shap, np_shap, df_rp, df_y = run_rp_pipeline(C=C, D=D, impute=impute, feats=feats, csl=csl)
    print(f'{log_head}Experiment completed --> {cur_result["Experiment_Name"]}')

    # Save the mean-absolute SHAP values
    out_sub_dir_path: str = f'{out_dir_path}SHAP/'
    os.makedirs(out_sub_dir_path, exist_ok=True)
    out_file_path: str = f'{out_sub_dir_path}{cur_result["Experiment_Name"]}.csv'
    df_shap.to_csv(out_file_path, index=False)

    # Save the raw SHAP values
    out_sub_dir_path: str = f'{out_dir_path}SHAP_RAW/'
    os.makedirs(out_sub_dir_path, exist_ok=True)
    out_file_path: str = f'{out_sub_dir_path}{cur_result["Experiment_Name"]}.npy'
    np.save(out_file_path, np_shap)

    # Save the cross-validation statistics
    out_sub_dir_path: str = f'{out_dir_path}Validation_Statistics/'
    os.makedirs(out_sub_dir_path, exist_ok=True)
    out_file_path: str = f'{out_sub_dir_path}{cur_result["Experiment_Name"]}.csv'
    df_rp.to_csv(out_file_path, index=False)

    # Save the predicted probabilities
    out_sub_dir_path: str = f'{out_dir_path}Predicted_Probabilities/'
    os.makedirs(out_sub_dir_path, exist_ok=True)
    out_file_path: str = f'{out_sub_dir_path}{cur_result["Experiment_Name"]}.csv'
    df_y.to_csv(out_file_path, index=False)

    # Concatenate the cur_result to all_results
    all_results.append(cur_result)
    print('*'*120)

    # Organize the current version of all_results as a pandas.DataFrame
    df_out: pd.DataFrame = pd.DataFrame.from_records(all_results)
    df_out.drop(columns=[col for col in df_out.columns if 'LIST' in col], inplace=True)

    # Save the current version of all_results
    out_file_path: str = f'{out_dir_path}Experiment_RP{"_Ablated" if ABLATION else ""}_{C}_{D}.xlsx'
    df_out.to_excel(out_file_path, index=False)
    optimize_width(out_file_path)
    print('Modeling result saved.')
    print('*'*120)
